# 4 Pannel NAM

In [5]:
####################################################
####################################################
####################################################
#
# Libraries
#

import numpy             as np
import datetime          as dt
import matplotlib        as mpl
import matplotlib.pyplot as plt
import ftplib            as ftplib
import urllib.request    as urllibreq
import datetime          as datetime
import os                as os
import platform          as platform
import socket            as socket
import xarray            as xr
import netCDF4           as nc4
import metpy             as metpy
import pathlib           as pathlib
import numpy             as np
import cftime            as cftime
import netCDF4           as nc4
import metpy             as metpy
import cartopy.crs       as ccrs
import cartopy.feature   as cfeature
import pandas            as pd



#
####################################################
####################################################
####################################################

## Time Settings

### Setting up Time Intervals

In [6]:
####################################################
####################################################
####################################################
#
# Model Time Intervals
#

time_interval_between_runs            =  6 # hours

time_between_output_steps             =  4 # hours

total_sumulation_time                 = 48 # hours

time_between_boundary_condition_feeds =  3 # hours

#
####################################################
####################################################
####################################################

### Timings for each run

The Realtime WRF is generated every 6 hr at best.  The model takes 3 hr to 

| Model Product Time (UTC) | Wallclock Start Time (UTC) |
|:------------------------:|:--------------------------:|
|        00 UTC            |        03 UTC              |
|        06 UTC            |        09 UTC              |
|        12 UTC            |        15 UTC              |
|        18 UTC            |        21 UTC              |

In [7]:
####################################################
####################################################
####################################################
#
# Identify Specific Run by Wall Clock Window
#

lag_hours = 3

current_datetime = datetime.datetime.utcnow()


current_datetime_lag3 = current_datetime - datetime.timedelta(hours=lag_hours)


if (current_datetime.day == current_datetime_lag3.day):
    if (current_datetime_lag3.hour < 6):
        fx_hour =  0
    elif (current_datetime_lag3.hour < 12):
        fx_hour =  6
    elif (current_datetime_lag3.hour < 18):
        fx_hour = 12
    else:
        fx_hour = 18

    model_start_datetime = datetime.datetime(year  = current_datetime_lag3.year,
                                             month = current_datetime_lag3.month, 
                                             day   = current_datetime_lag3.day, 
                                             hour  = fx_hour)     
else:
    fx_hour = 18
    model_start_datetime = datetime.datetime(year  = current_datetime_lag3.year,
                                             month = current_datetime_lag3.month, 
                                             day   = current_datetime_lag3.day, 
                                             hour  = fx_hour)

    
model_end_datetime        = model_start_datetime + datetime.timedelta(hours=total_sumulation_time)
model_product_dates       = pd.date_range(model_start_datetime, periods=5, freq="12H")
model_product_dates_hours = model_product_dates[1:] - model_product_dates[0]


print("           Current Time ", current_datetime)
print("NAM Forecast Start Time ", model_start_datetime)
print("  NAM Forecast End Time ", model_end_datetime)

print(" ")
print("NAM Forecast Prod Times ", model_product_dates)




#
# Burn Current Time to File in WRF Root Directory
#

model_start_date_YYYYMMDDHH         = model_start_datetime.strftime("%Y%m%d%H")
model_start_date_YYYY_MM_DD_HH      = model_start_datetime.strftime("%Y-%m-%d_%H")
model_start_date_YYYY_MM_DD_HH00UTC = model_start_datetime.strftime("%Y-%m-%d %H00 UTC")


model_thredds_retrieval_date   = model_start_datetime.strftime("%Y%m%d_%H00")

nam_opendap_url = "https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NAM/CONUS_40km/conduit/NAM_CONUS_40km_conduit_" + model_thredds_retrieval_date + ".grib2"
                  #https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NAM/CONUS_40km/conduit/NAM_CONUS_40km_conduit_20220124_0000.grib2
print(" ")

print(nam_opendap_url)

#
####################################################
####################################################
####################################################

           Current Time  2022-02-01 22:24:33.547544
NAM Forecast Start Time  2022-02-01 18:00:00
  NAM Forecast End Time  2022-02-03 18:00:00
 
NAM Forecast Prod Times  DatetimeIndex(['2022-02-01 18:00:00', '2022-02-02 06:00:00',
               '2022-02-02 18:00:00', '2022-02-03 06:00:00',
               '2022-02-03 18:00:00'],
              dtype='datetime64[ns]', freq='12H')
 
https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NAM/CONUS_40km/conduit/NAM_CONUS_40km_conduit_20220201_1800.grib2


## Crack open GRIB array with Xarray

In [8]:
####################################################
####################################################
####################################################
#
# Crack open the UCAR thredds NOMADS array.
#

nam_model = xr.open_dataset(nam_opendap_url)

display(nam_model)

#
####################################################
####################################################
####################################################

Note:Caching=1
Error:curl error: Couldn't resolve host name
curl error details: 


OSError: [Errno -68] NetCDF: I/O failure: b'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NAM/CONUS_40km/conduit/NAM_CONUS_40km_conduit_20220201_1800.grib2'

## Panel Displays

500-hPa Vorticity/Heights

* 500-hPa Absolute Vorticity
* 500-hPa Isobaric Heights

Thickness / MSLP

* Mean Sea Level Pressure
* 1000-500 hPa Thickness

850-hPa Heights and Humidity

* 850-hPA Heights
* 850-hPA Relative Humidity

Precipitation / Vertical Velocity

* 12-hr Precipitation
* 700-hPa Vertical Velocity


## Fetch Data for Pannel Displays

In [5]:
####################################################
####################################################
####################################################
#
# Extract Map Fields
#

# 500-hPa Heights & Vorticity

vorticity_500 = nam_model[ "Absolute_vorticity_isobaric"].sel(isobaric  = 50000).sel(time4 = model_product_dates)
heights_500   = nam_model["Geopotential_height_isobaric"].sel(isobaric1 = 50000).sel(time4 = model_product_dates)

# MSLP - Thickness

mslp      = nam_model["Pressure_reduced_to_MSL_msl"].sel(time4 = model_product_dates)
thickness = heights_500 - nam_model["Geopotential_height_isobaric"].sel(isobaric1 = 100000).sel(time4 = model_product_dates)

# 850-hPa Heights & RH

humidity_850  = nam_model[  "Relative_humidity_isobaric"].sel(isobaric1 = 85000).sel(time4 = model_product_dates)
heights_850   = nam_model["Geopotential_height_isobaric"].sel(isobaric1 = 85000).sel(time4 = model_product_dates)

# Precip & 700-hPa VVel



vvel_700   = nam_model["Vertical_velocity_pressure_isobaric"].sel(isobaric1 = 85000).sel(time4 = model_product_dates)

precip = nam_model["Total_precipitation_surface_3_Hour_Accumulation"]

for k in range(1,precip.time3.size) :
    precip.values[k,:,:] = precip.values[k-1,:,:]+precip.values[k,:,:]

precip = precip.sel(time3 = model_product_dates[1:]) 
precip.values[1:,:,:] = precip.values[1:,:,:]-precip.values[:-1,:,:]


    
#
####################################################
####################################################
####################################################

KeyError: 'time4 is not a valid dimension or coordinate'

In [ ]:
precip

In [ ]:
np.arange(0,17,4)